In [126]:
import pandas as pd
import requests
import os

# Descargar archivo si no existe
file_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"
file_name = "yellow_tripdata_2022-01.parquet"


# Leer archivo Parquet con pandas
print(" Leyendo archivo Parquet...")
df = pd.read_parquet(file_name, engine="pyarrow")

# Calcular duración del viaje en minutos
df["duracion_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[(df["duracion_min"] > 1) & (df["duracion_min"] < 180)]  # eliminar extremos

# Calcular promedio
duracion_media = df["duracion_min"].mean()


print(f"\n Duración promedio del viaje: {duracion_media:.2f} minutos")

 Leyendo archivo Parquet...

 Duración promedio del viaje: 12.91 minutos


In [110]:
import dask
dask.config.set(scheduler='single-threaded')



In [130]:
import dask
import dask.dataframe as dd
import requests
import os

# Forzar Dask a usar un solo hilo (sin cluster)
dask.config.set(scheduler='single-threaded')

# ------------------------
# Paso 1: Descargar archivo Parquet si no existe
# ------------------------
file_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"
file_name = "yellow_tripdata_2022-01.parquet"



# ------------------------
# Paso 2: Leer archivo Parquet con Dask
# ------------------------
print("📊 Leyendo archivo con Dask...")
df = dd.read_parquet(file_name, engine="pyarrow", split_row_groups=True)

# ------------------------
# Paso 3: Tomar muestra pequeña para evitar cuelgues
# ------------------------
df_sample = df.sample(frac=0.001)

# ------------------------
# Paso 4: Calcular duración del viaje
# ------------------------
df_sample["duracion_min"] = (df_sample["tpep_dropoff_datetime"] - df_sample["tpep_pickup_datetime"]).dt.total_seconds() / 60
df_sample = df_sample[(df_sample["duracion_min"] > 1) & (df_sample["duracion_min"] < 180)]

# ------------------------
# Paso 5: Calcular media
# ------------------------
duracion_promedio = df_sample["duracion_min"].mean().compute()

print(f"\n Duración promedio del viaje (0.1% muestra): {duracion_promedio:.2f} minutos")


📊 Leyendo archivo con Dask...


C:\Users\jorge\anaconda3\Lib\site-packages\dask\base.py:1541: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(



 Duración promedio del viaje (0.1% muestra): 13.11 minutos
